In [53]:
# Importing necessary Libraries

import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urlencode


In [54]:
# Defining the API for the Request (Getting the City ID)
city = "Chennai"

url = "https://locator-service.api.bbci.co.uk/locations?"+ urlencode({
       "api_key":"AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv",
        "s":city,
        "stack":"aws",
        "locale":"en",
        "filter":"international",
        "place-types":"settlement,airport,district",
        "order":"importance",
        "a":'true',
        'format':'json'
    })


In [55]:
# Sending the request

result = requests.get(url).json()
city_id = result["response"]["results"]["results"][0]["id"]

In [56]:
print(city_id)

1264527


In [57]:
# Scrapping the webpage of that city

webPage = requests.get("https://www.bbc.com/weather/"+city_id)

In [58]:
# Parsing the Request to html content

soup = BeautifulSoup(webPage.content,'html.parser')

In [59]:
# Extracting the High and Low temp.

dailyHigh = soup.find_all('span',attrs={
    'class':"wr-day-temperature__high-value"
})

dailyLow = soup.find_all('span',attrs={
    'class':"wr-day-temperature__low-value"
})

In [60]:
# formatting the High Temp. data
high = []
for oneDay in dailyHigh:
    high.append(oneDay.text.split()[0][:-1])
high.append(None)

In [61]:
# formatting the Low Temp. data

low = []
for oneDay in dailyLow:
    low.append(oneDay.text.split()[0][:-1])

In [62]:
# Generating the dates

date = datetime.datetime.now()
date = int(str(date).split()[0].split("-")[-1])

dateList = []
for i in range(14):
    dateList.append(f"2023-06-{date}")
    date += 1

In [63]:
# Creating the DataFrame

df = {"Date":dateList,
      "Low":low,
      "High":high}
df = pd.DataFrame(df)
print(df)

          Date Low  High
0   2023-06-16  29    39
1   2023-06-17  27    35
2   2023-06-18  26    33
3   2023-06-19  25    34
4   2023-06-20  25    35
5   2023-06-21  25    34
6   2023-06-22  26    34
7   2023-06-23  26    34
8   2023-06-24  26    34
9   2023-06-25  26    34
10  2023-06-26  26    35
11  2023-06-27  26    34
12  2023-06-28  27    33
13  2023-06-29  26  None


In [64]:
# Saving the DataFrame to csv file
df.to_csv("BBC-Weather.csv",index= False)